<a href="https://colab.research.google.com/github/omerbakler/ImageProccessOB/blob/main/OmerBakler_Notebook_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello


This is my notebook from "Image Proccessing" Course, at the Hebrew University of Jerusalem Institute, Robert H. Smith Faculty of Agriculture. The course was taken in the fall semester of 2022-23 year. 
This notebook contains essential image processing tools from the very basic to machine learning. For any questions and notes please contact.

# Basic numpy

In [ ]:
x = np.array([1,2,3,4,5]) # creating an array
x.shape    #return a tuple of array dimensions
x.size     #return number of elements in the array
x.dtype    #return the data type of the elements in the array
x = np.array([1, 2], dtype=np.int64)     #force a specific data type
# Access the 0th row of a:
x_row0 = x[0,:]

Slicing: 

In [ ]:
# Access the 0th row of a:
x_row0 = x[0,:]

# Access the 0th column of a
a_col0 = a[:,0]

# arange puts all number between 2 values in the tuple
# reshape changes the size of the array to the dimensions in the tuple
b = np.arange(1,17).reshape((4,4))

Arithmatic operations:

In [ ]:
print("Original x: ")
print(x)
print("\nx+1")
print(x + 1)
print("\nx*4.5")
print(4.5*x)
print("\nx/2.0")
print(x/2.0)

print(x - y)
print('\n', np.subtract(x, y))

print(x * y)
print('\n',np.multiply(x, y))

print(x / y)
print('\n',np.divide(x, y))

print(np.sqrt(x))

print(np.power(x,2))

Other useful functions:

In [ ]:
np.concatenate((x,y), axis = 0)    #adding to arrays together, on the axis given.
np.transpose(result)     #switching rows and columns
x.dot(y)
np.dot(x, y)     #calculating dot product
np.matmul(x,y)   # matrix multiplication

Boolean mask:

In [ ]:
import numpy as np
array = np.array(range(20)).reshape((4,5))
output = array > 10
print(output)
print(array[output])

[[False False False False False]
 [False False False False False]
 [False  True  True  True  True]
 [ True  True  True  True  True]]
[11 12 13 14 15 16 17 18 19]


copy:

In [ ]:
import copy
y = copy.deepcopy(x)

y = x.copy()      #builtin numpy copy method

# Basic images tools

Imprtant Libs to import:

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import skimage
import skimage.io as io
import cv2 

Mount drive and creating working folder:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/71254_2023/01_Lectures/Class02'

Reading and displaing an image:

In [ ]:
image = io.imread(image_path)            #reading an image using skimage
image = io.imread(fname=f"{folder_path}/image.jpeg")   #read and display image the same time
plt.figure(figsize = (5,5))              #setting image sizes to show
plt.imshow(image)                        #displaying the image
plt.axis('off')                          #displaing or not the axis bars
image = image.astype(np.float64) / 255   #Normalizing the image data type
plt.colorbar()                           #display colorbar next to image

Excluding a specific channel from an RGB image. If it's BGR with cv2, the order should be reversed.

In [ ]:
def rgb_exclusion(image, channel):
    out = image.copy()
    if channel == 'R':
        out[:, :, 0] = 0
    elif channel == 'G':
        out[:, :, 1] = 0
    elif channel == 'B':
        out[:, :, 2] = 0

    return out

no_green = rgb_exclusion(img, 'G')

b,g,r = cv2.split(image)     #splitting the image into 3 different arrays using cv2.

merged_image = cv2.merge([r, g, b])   #stacked image back together, the order is rgb because it is cv2.

Convert image to grayscale:

In [ ]:
gray_image = skimage.color.rgb2gray(image)   #convert to gray scale with built in skimage function
plt.imshow(gray_image, cmap='Greys_r')       #displaying the image, using gray color map.

Displaying several images organized:

In [ ]:
plt.subplot(1,2,1)
plt.imshow(green_channel)

plt.subplot(1,2,2)
plt.imshow(blue_channel)

'imshow' function for displaying multiple images side by side. Input is a list of the images and list of the titles for the images.

In [ ]:
def imshow_all(images, titles=None):
    images = [img_as_float(img) for img in images]

    if titles is None:
        titles = [''] * len(images)
    vmin = min(map(np.min, images))
    vmax = max(map(np.max, images))
    ncols = len(images)
    height = 5
    width = height * len(images)
    fig, axes = plt.subplots(nrows=1, ncols=ncols,
                             figsize=(width, height))
    for ax, img, label in zip(axes.ravel(), images, titles):
        ax.imshow(img, vmin=vmin, vmax=vmax, cmap = 'gray')
        ax.set_title(label)

# Multidimensional arrays and spectral indices

Relevant libs:

In [ ]:
from spectral import imshow, view_cube, ndvi
import spectral.io.envi as envi

Files that was taken from hyperspectral camera:

In [ ]:
dark_ref = envi.open(f'{hdr_path}/DARKREF_1341.hdr', f'{hdr_path}/DARKREF_1341.raw')
white_ref = envi.open(f'{hdr_path}/WHITEREF_1341.hdr', f'{hdr_path}/WHITEREF_1341.raw')
data_ref = envi.open(f'{hdr_path}/1341.hdr', f'{hdr_path}/1341.raw')

white_nparr = np.array(white_ref.load())    #reading into arrays
dark_nparr = np.array(dark_ref.load())
data_nparr = np.array(data_ref.load())

Correcting the image with the correction formula:

In [ ]:
corrected_nparr = np.divide(
    np.subtract(data_nparr, dark_nparr),
    np.subtract(white_nparr, dark_nparr))

imshow(corrected_nparr, (100, 100, 100))

Creating an array with the wavelength's of image:

In [ ]:
bands = np.genfromtxt(f'{hdr_path}/bands.csv', delimiter=',')

Function to show the spectral footprint of a specific pixel, given the x,y labels of the pixel and the bands:

In [ ]:
def plot_pixel(img,pixel_y,pixel_x,bands):

  leaf_pixel = img[
      pixel_y:pixel_y+1,
      pixel_x:pixel_x+1,
      :]

  leaf_pixel_squeezed = np.squeeze(leaf_pixel)

  plt.plot(bands, leaf_pixel_squeezed)
  plt.title('Spectral Footprint\n(Pixel {},{})'.format(
      pixel_x, pixel_y),fontsize=16)
  plt.xlabel('Wavelength',fontsize=14)
  plt.ylabel('Reflectance',fontsize=14)
  plt.show()

Do the same, with several pixels:

In [ ]:
def plot_several_pixel(img,pixels_tuples_array,bands):
  for pixel_xy in pixels_tuples_array:
    leaf_pixel = img[ 
      pixel_xy[1]:pixel_xy[1]+1, #y pixel
      pixel_xy[1]:pixel_xy[1]+1, #x pixel
      :]
    leaf_pixel_squeezed = np.squeeze(leaf_pixel) #squeeze

    plt.plot(bands, leaf_pixel_squeezed, label =f"x={pixel_xy[1]}, y={pixel_xy[1]}"  )
  plt.title('Spectral signature',fontsize=16)
  plt.xlabel('Wavelength',fontsize=14)
  plt.ylabel('Reflectance',fontsize=14)
  plt.legend()
  plt.show()

Flip the image:

In [ ]:
flipped = np.fliplr(rgb_tomato_wall)

# Index calculation

 $$NDVI = \frac{(NIR - RED)}{(NIR + RED)}$$

Calculating image NDVI:

In [ ]:
red_channel = corrected_nparr[:,:,87] # 651.92 nm
nir_channel = corrected_nparr[:,:,140] # 810.86 nm

NDVI = (nir_channel - red_channel) / (nir_channel + red_channel) # calc manually
vi = ndvi(corrected_nparr, 87, 140) # using the ndvi function from spectral to calc, arguments: image, red band number, nir band number

 $$ExG = 2*GREEN - RED - BLUE$$

In [ ]:
# process the image for calculating the ExG index
import cv2
b,g,r= cv2.split(img) # split the images into channels
np.seterr(invalid='ignore') # ignore 0/0 when dividing

# function to normalize each channel
def normalize(img):
  return (img - np.min(img)) / (np.max(img) - np.min(img))

# you can also use img_as_float - uncomment to check
from skimage.util import img_as_float
image = img_as_float(img)

r,g,b = normalize(r), normalize(g), normalize(b) # normalizing all channels

exg = 2*g-r-b # calculating the index
exg_mean = np.nanmean(exg) # calc the mean of the image

# display
plt.imshow(exg, cmap='RdYlGn')
plt.colorbar()
plt.title(f'ExG Index, mean: {exg_mean:.2f}') # notice the mean printed in the title, and only 2 digits after decimal point (using a f-string). Read more about f-strings here: https://realpython.com/python-f-strings/

# Reading multiple images from a folder using glob

Reading all path's into a list:

In [ ]:
import glob
import skimage.io as io

path_list = glob.glob(f'{folder_path}/images/*.*') #Rerurns a list of file names
print(path_list)  #Prints the list containing file names

Iterating over the path list and reand each image into a list of images:

In [ ]:
image_list=[]  # Empty list to store images from the folder.
path = "images/test_images/*.*"
for path in path_list:   #Iterate through each file in the list using for
    print(path)     # just stop here to see all file names printed
    img = io.imread(path)  # now, we can read each file since we have the full path
    image_list.append(img)  #Create a list of images (not just file names but full images)

# Downloading images

Function to create a new folder for images if doesn't exist:

In [ ]:
import os # import os lib

def createDir(path):
  doesExist = os.path.exists(path) # checks whether the specified path exists
  if not doesExist:
    os.makedirs(path) # create path, since it doesn't exist
    print("The new directory was created!")

Download a specific image from a url

In [ ]:
import requests # lib to request image from web
import shutil # lib to save the image locally

def downloadImageFromURL (url='', destination_image_path=''):

  # The method will take in two parameters, the url variable you created earlier, and stream: True.
  # by adding this second argument in guarantees no interruptions will occur when the method is running.
  res = requests.get(url, stream = True) 

  if res.status_code == 200:
      with open(destination_image_path,'wb') as f:
          shutil.copyfileobj(res.raw, f)
      print('Image sucessfully Downloaded: ', destination_image_path)
  else:
      print('Image Couldn\'t be retrieved')

In [ ]:
image_url = '/url.jpg'
path_to_save_image = f'{images_path}/image.jpg'
downloadImageFromURL(image_url, path_to_save_image)

Downloading images directly from google images search:

In [ ]:
# first uninstall the lib that colab has (enter 'y' to uninstall, and restart runtime)
!pip uninstall google_images_download

# then install the updated library
!pip install git+https://github.com/Joeclinton1/google-images-download.git

In [ ]:
# imports libs (again, since runtime was restarted)
import numpy as np
from matplotlib import pyplot as plt
import cv2
import skimage.io as io

# import the newly installed library
from google_images_download import google_images_download

# function to download images from Google Image Search
def downloadImagesFromGoogle(words_to_search='', number_of_imgs=3):

  response = google_images_download.googleimagesdownload() #instantiate the class
  arguments = {"keywords":words_to_search,
              "limit":number_of_imgs,"print_urls":False}
  paths = response.download(arguments)  
  print(paths) #print complete paths to the downloaded images

In [ ]:
# calling our function above (images will be downloaded to the 'downloads' folder on the left)
downloadImagesFromGoogle('corn plant,watermelon plant', 3)

In [ ]:
import glob
watermelon_paths = glob.glob('/content/downloads/watermelon plant/*.*') # get the paths of the watermelon images

# read images into a list
img_list = []
for path in watermelon_paths:
  img = io.imread(path)
  img_list.append(img)

In [ ]:
# displaying the downloaded images
fig, ax=plt.subplots(ncols=len(img_list), nrows=1, figsize=(8, 8))
plt.tight_layout() # nicer layout

# loop to plt.imshow all the images in one row
for i in range(len(img_list)):
  ax[i].imshow(img_list[i])
  ax[i].set_title(f'Image {i+1}')

Download image with one line using wget.

In [ ]:
img_filename = 'img_example.jpg'

# using wget(a command-line tool that makes it possible to download files) - first the direct link, and then optional, you can pass a filename.
!wget https://www.dropbox.com/s/n3xts37k5sqfgok/street.jpg -O {img_filename}

# Histogams

Using opencv:


```
hist = cv.calcHist(images, channels, mask, histSize, ranges[, hist[, accumulate]])
```
Let's familiarize with the function and its parameters :

1. **images** : it is the source image of type uint8 or float32. it should be given in square brackets, ie, "[img]".
2. **channels** : it is also given in square brackets. It is the index of channel for which we calculate histogram. For example, if input is grayscale image, its value is [0]. For color image, you can pass [0], [1] or [2] to calculate histogram of blue, green or red channel respectively.
3. **mask** : mask image. To find histogram of full image, it is given as "None". But if you want to find histogram of particular region of image, you have to create a mask image for that and give it as mask.
4. **histSize** : this represents our BIN count. Need to be given in square brackets. For full scale, we pass [256].
5. **ranges** : this is our RANGE. Normally, it is [0,256]. (the hist and accumalte inside the ranges, enables to compute a single histogram from several sets of arrays.)


One channel histogram:

In [ ]:
# computing the histogram of the red channel of image
image = image_list[0].copy()
hist = cv2.calcHist([image],[0],None,[256],[0,256]) # our image, first channel(0=red), no mask, 256 bins, range 0-255

# plot the above computed histogram
plt.plot(hist, color='r') # red color for the line
plt.title('Image histogram for the red channel of our watermelon image', y=1.03) # the y makes the title a bit higher
plt.xlabel("Pixel Value")
plt.ylabel("Count")
plt.show()

Multiple channels histogram:

In [ ]:
# computing the histogram of all channels of our watermelon image
img = img_list[0].copy()
  
# plot the above computed histogram
colors = ('r','g','b') # for the line color

for i,color in enumerate(colors):
  hist = cv2.calcHist([img],[i],None,[256],[0,256]) # our image, channel(0/1/2), no mask, 256 bins, range 0-255
  plt.plot(hist, color = color) # r/g/b color for each line
plt.title('Image histogram (RGB) of our watermelon image', y=1.03) # the y makes the title a bit higher
plt.xlabel("Pixel Value")
plt.ylabel("Count")
plt.show()

Histogram using plt, one channel:

In [ ]:
hist_with_plt = plt.hist(img[:,:,1].flatten(), bins = 256, color='g')
plt.title('Image histogram (green layer) of our watermelon image', y=1.03) # the y makes the title a bit higher
plt.xlabel("Pixel Value")
plt.ylabel("Pixel Count")
plt.show()

Multiple channels:

In [ ]:
colors = ('r','g','b') # for the line color

hist = plt.hist(img.flatten(), bins = 256, color = 'orange') # plotting the total histogram

#loop to plot for each channel
for i,color in enumerate(colors):
  hist = plt.hist(img[:, :, i].flatten(), bins = 256, color = color, alpha = 0.5)

# some more details for the figure
plt.title('Image histogram (RGB) of our watermelon image', y=1.03) # the y makes the title a bit higher
plt.xlabel("Pixel Value")
plt.ylabel("Pixel Count")
plt.legend(['Total', 'Red Channel', 'Green Channel', 'Blue Channel']) # adding a legend
plt.show()

# Operaions on images

### Addition

In [ ]:
paths_list = glob.glob(f'{images_path}/addition/*.*') # paths to list
images_list = [io.imread(path) for path in paths_list] # read all images into a list in one line

for img in images_list:
  plt.title(img.shape)
  plt.imshow(img)
  plt.show()

Sometimes we need to resize one of the images to fit the other:

In [ ]:
# resize the large one (2) to the smaller one (1)
resized2 = cv2.resize(images_list[1], (481,209), interpolation = cv2.INTER_AREA)
plt.imshow(resized2)
plt.title(resized2.shape)

Function to add 2 images together:

In [ ]:
if images_list[0].shape == resized2.shape:
  added_img = images_list[0] + resized2
  plt.imshow(added_img)
  plt.title('Two images added together')
else:
  print('Shapes are different, cannot add.')

### Multiplication:

For multiplication, we first need to convert the image to float:

In [ ]:
dark_img_float = img_as_float(dark_img.copy())

This just simple array function, can multiply by any number:

In [ ]:
multiplyed = dark_img_float*2

### Division and Subtraction:

Getting the size of the image we want:

In [ ]:
objects = plt.imread(objects_path)
no_objects = plt.imread(no_objects_path)
size = no_objects.shape
print(size)

Changing the other image to the same size and converting to float:

In [ ]:
resized_objects = cv2.resize(objects, (318,300), interpolation = cv2.INTER_AREA)

resized_objects_float = img_as_float(resized_objects)
no_objects_float = img_as_float(no_objects)

Dividing and subtracting the images:

In [ ]:
divided_image = resized_objects_float/no_objects_float
plt.imshow(divided_image)
plt.title("divided image")
plt.show()

subtracted_image = no_objects_float - resized_objects_float
plt.imshow(subtracted_image)
plt.title("subtracted image")

# Filters

### Mean filter:

This is a mean of a 3X3 square:

In [ ]:
mean_kernel = np.full((3, 3), 1/9)

Using the mean kernel array to apply kernel convulation:

In [ ]:
import scipy.ndimage as ndi
mean_image = ndi.correlate(image,mean_kernel)

### Gaussian filter:

Simple gaussian filter with skimage filters:

In [ ]:
from skimage import filters
sigma = 1 # for the gaussian filter
smooth = filters.gaussian(image, sigma)

The Gaussian filter returns a float image, regardless of input. Cast to float so the images have comparable intensity ranges.

In [ ]:
from skimage import img_as_float
float_image = img_as_float(image)

Gaussian using OpenCV:
Agruments: image, kernel size, sigma values, border type (padding, etc.)
cv2.BORDER_CONSTANT adds a constant color border


In [ ]:
gaussian_using_cv2 = cv2.GaussianBlur(image, (3,3), 0, borderType=cv2.BORDER_CONSTANT) 

### Unsharp mask

enhanced image = original + amount * (original - blurred)

In [ ]:
gaussian_img = gaussian(img, sigma=2, mode='constant', cval=0.0)
enhanced_image = img + (img - gaussian_img)*1

### Vertical and horizontal kernel

In [ ]:
# lets create a vertical kernel
vertical_kernel = np.array([
    [-1],
    [ 0],
    [ 1],
])
horizontal_kernel = (vertical_kernel.T)

# convolve our above image
gradient_vertical = ndi.correlate(pixelated.astype(float),
                                  vertical_kernel)
gradient_horizontal = ndi.correlate(pixelated.astype(float),
                                  horizontal_kernel)

Magnitude calculation:

In [ ]:
mag = np.sqrt((gradient_horizontal)**2+(gradient_vertical)**2)

### Sobel edge detector

In [ ]:
img_gradient = filters.sobel(img)

### Canny edge detector

The Process of Canny edge detection algorithm can be broken down to 5 different steps:

1. Apply Gaussian filter to smooth the image in order to remove the noise
2. Find the intensity gradients of the image
3. Apply non-maximum suppression to get rid of spurious response to edge detection
4. Apply double threshold to determine potential edges (supplied by the user)
5. Track edge by hysteresis: Finalize the detection of edges by suppressing all the other edges that are weak and not connected to strong edges

In [ ]:
canny_edge = cv2.Canny(img, 195, 200)  #Supply Thresholds 1 and 2 

### Median filter

A useful filter to blur the image without losing the edges. Works like a mean filter but applies the median value instead of mean value.

In [ ]:
#Disk creates a circular structuring element, similar to a mask with specific radius
from skimage.morphology import disk
neighborhood = disk(radius=1)  # "structuring element", matrix of ones though corners are zeroes

median = filters.rank.median(pixelated, neighborhood) # applying the median filter, needs 8 bit, not float.

### Bilateral filter

In [ ]:
from skimage import img_as_ubyte

# d - diameter of each pixel neighborhood used during filtering
# sigmaCOlor - Sigma of grey/color space. 
# sigmaSpace - Large value means farther pixels influence each other (as long as the colors are close enough)
bilateral_using_cv2 = cv2.bilateralFilter(img, 5, 20, 100, borderType=cv2.BORDER_CONSTANT)

### Non-local means (NLM) filter

Works well for random gaussian noise but not as good for salt and pepper

The non-local means algorithm replaces the value of a pixel by an average of a selection of other pixels values: small patches centered on the other pixels are compared to the patch centered on the pixel of interest, and the average is performed only for pixels that have patches close to the current patch. 

### Total variation filter (TVF)

In [ ]:
import cv2
from skimage import io, img_as_float
from skimage.restoration import denoise_tv_chambolle
from matplotlib import pyplot as plt

denoise_img = denoise_tv_chambolle(img, weight=0.1, eps=0.0002, n_iter_max=200, multichannel=False)

### Interpolation

In [ ]:
width, height = lion_img.size

im_nearest = lion_img.resize((width*4 ,height), Image.NEAREST)

im_bilinear = lion_img.resize((width*4 ,height), Image.BILINEAR) 

im_bicubic = lion_img.resize((width*4 ,height), Image.BICUBIC) 

imshow_all([im_nearest,im_bilinear,im_bicubic], ['nearest','bilinear','bicubic'])

### Fourier transforn

Fourier transform converts an image (or any signal) from spatial (or time) domain to frequency domain. Here's an example with Python.

In [ ]:
#Output is a 2D complex array. 1st channel real and 2nd imaginary
#For fft in opencv input image needs to be converted to float32
dft = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)

#Rearranges a Fourier transform X by shifting the zero-frequency 
#component to the center of the array.
#Otherwise it starts at the top left corner of the image (array)
dft_shift = np.fft.fftshift(dft)

# Circular HPF mask, center circle is 0, remaining all ones
#Can be used for edge detection because low frequencies at center are blocked
#and only high frequencies are allowed. Edges are high frequency components.
#Amplifies noise.

rows, cols = img.shape
crow, ccol = int(rows / 2), int(cols / 2)

mask = np.ones((rows, cols, 2), np.uint8)
r = 80
center = [crow, ccol]
x, y = np.ogrid[:rows, :cols]
mask_area = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= r*r
mask[mask_area] = 0


# apply mask and inverse DFT
fshift = dft_shift * mask

fshift_mask_mag = 2000 * np.log(cv2.magnitude(fshift[:, :, 0], fshift[:, :, 1]))

f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

### Removing holes

Oprion 1:

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8)) # define a kernel (change size if needed)
res = cv2.morphologyEx(img_as_ubyte(img),cv2.MORPH_OPEN,kernel) # applying the kernel to our binary (make sure it's not a boolean array)


Option 2:

In [ ]:
from scipy import ndimage as nd
closed_mask = nd.binary_closing(mask, np.ones((5,5)))

# Segmentation

### Histogram based segmentation

Thresholding is a type of image segmentation, where we change the pixels of an image to make the image easier to analyze. In thresholding, we convert an image from colour or grayscale into a binary image, i.e., one that is simply black and white. Most frequently, we use thresholding as a way to select areas of interest of an image, while ignoring the parts we are not concerned with.

First we want to see the histogram of the image.

In [ ]:
hist_with_plt = plt.hist(img.flatten(), bins = 256, color='gray')

Now we have the values of interest in the image. We can apply manual thresholding. Example (the value is 0.55):

In [ ]:
# Manual thresholding by setting threshold value to numpy array
# After thresholding we will get a binary image.

mask = (img > 0.55)

masked_img[mask == 0] = 0 # if mask is False, then zero the pixel in the original image
masked_img[mask != 0] = rgb_img[mask != 0] # if mask is True, then leave the original image

We can do it also with openCV:

In [ ]:
ret1, thresh1 = cv2.threshold(img, 0.55, 1, cv2.THRESH_BINARY)
plt.imshow(thresh1, cmap='gray')

### Otsu's method

A method to apply automatic thresholding. Particularly useful for situations where the grayscale histogram of an image has two peaks that correspond to background and objects of interest.

In [ ]:
from skimage import img_as_ubyte
ret2, thresh2 = cv2.threshold(img_as_ubyte(img),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# ret2 is the value the method has picked.
# thresh2 is the image after thresholding.

plt.imshow(thresh2, cmap='gray')
plt.title('Our mask, using cv2+otsu')

Also possible with skimage:

In [ ]:
import skimage
from skimage import filters

# perform automatic thresholding
t = skimage.filters.threshold_otsu(img)
print("Found automatic threshold t=", t, f' ({255*t})\n')

# create a binary mask with the threshold found by Otsu's method
binary_mask = img > t

# display
plt.imshow(binary_mask, cmap='gray')

Otsu's is also useful for multiple threshold. It is usually represent numerous segments of interest in the image.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from skimage.filters import threshold_multiotsu
import cv2

# Apply multi-Otsu threshold 
thresholds = threshold_multiotsu(img, classes=4)
print(thresholds)

We can use it to classify the image into parts of interest with diffrent colors, but all the pixels within the same range will be the same.

In [ ]:
# Digitize (segment) original image into multiple classes.
# np.digitize assign values 0, 1, 2, 3, ... to pixels in each class.
regions = np.digitize(img, bins=thresholds)
plt.imshow(regions)

We can choose specific region:

In [ ]:
# Lets take a look at region 3
plt.imshow(regions==3, cmap='gray')

### Texture

Entropy:

In a nutshell, the entropy function gives a value that represents level of complexity in a certain section of an image. The resulting values are of course subject to the initial structuring element we chose.


In [ ]:
from skimage.stats import entropy
from skimage.morphology import disk

entropy_img = entropy(img, disk(7)) # play with the disk size to get better results
plt.imshow(entropy_img)

ModuleNotFoundError: ignored

We can look at the histogram of the entropy image:

In [ ]:
#Now let us use otsu to threshold high vs low entropy regions.
plt.hist(entropy_img.flat, bins=100, range=(0,6))  #.flat returns the flattened numpy array (1D)
plt.show()
thresh = threshold_otsu(entropy_img)
print(thresh)

#Now let us binarize the entropy image 
binary = entropy_img <= thresh

#display
plt.imshow(binary)

We want to get rid of holes inside the result:

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8,8)) # define a kernel (change size if needed)
res = cv2.morphologyEx(img_as_ubyte(img),cv2.MORPH_OPEN,kernel) # applying the kernel to our binary (make sure it's not a boolean array)


Now we invert the image:

In [ ]:
new_mask = cv2.bitwise_not(res)

We multiply by the image to see only the parts we intrested in. The mask is binary so multiplying will leave only the intresting parts.

In [ ]:
plt.imshow(new_mask*img, cmap='gray')

### Color Spaces

Using the different color segments to divide the image. First we need to convert from RGB to HSV.

In [ ]:
# convert to hsv color space
hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) # convert rgb to hsv

mask = cv2.inRange(hsv, (100,90,90), (120,255,255)) # mask out the blue balls. range of hsv. from hsv1 to hsv2. 
# enter the link to see the hsv color space
# https://stackoverflow.com/questions/10948589/choosing-the-correct-upper-and-lower-hsv-boundaries-for-color-detection-withcv/48367205#48367205
# we chose only h and s. v we made same as s

#mask = cv2.inRange(hsv, (0,0,180), (180,70,255)) # White

Usually there will be holes in the spaces beacuse colors are not definite. We want to get rid of thos holes.

In [ ]:
from scipy import ndimage as nd
closed_mask = nd.binary_closing(mask, np.ones((5,5)))

### Watershed

Watershed segmentantion is one of the most popular algorithm for image segmentation, normally used when we want to resolve one of the most difficult operations in image processing — separating similar objects in the image that are touching each other.

In essence, we partition the image into two different sets:

1. catchment basins — dark areas; The group of connected pixels with the same local minimum.
2. watershed lines — Lines that divide one catchment area from another.

Here is the function for watershed segmentation.

In [ ]:
# your code goes here

import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy import ndimage
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage import io, img_as_ubyte
import imutils

def WSsegment(image_path,min_dist):
  img = cv2.imread(image_path) # read image
  
  if img.dtype != 'uint8':
    img_rgb = img_as_ubyte(img)
  
  if len(img.shape) == 1:
    gray = img.copy()
  else:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # bgr to rgb
    shifted = cv2.pyrMeanShiftFiltering(img, 21, 51)
    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
  
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
  
  D = ndimage.distance_transform_edt(thresh)

  localMax = peak_local_max(D, indices=False, min_distance=min_dist, labels=thresh)

  markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
  labels = watershed(-D, markers, mask=thresh)

  total_area = []

  for i,label in enumerate(np.unique(labels)):
    if label == 0:
      continue
    mask = np.zeros(gray.shape, dtype="uint8")
    mask[labels == label] = 255

    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    ((x, y), r) = cv2.minEnclosingCircle(c)
    cv2.drawContours(img_rgb, cnts, -1, (0,255,0), 1) # draw the counters
    plt.imshow(img_rgb)

### Voronoi

First, we import the relevant lib and change to GPU.

In [ ]:
import pyclesperanto_prototype as cle
from skimage import exposure, img_as_ubyte

gray = skimage.color.rgb2gray(image)

#Normalize then scale to 255 and convert to uint8 - using skimage
cells_8bit = img_as_ubyte(gray)

ModuleNotFoundError: ignored

### K-means

K-Means clustering is an unsupervised machine learning algorithm that aims to partition N observations into K clusters in which each observation belongs to the cluster with the nearest mean. A cluster refers to a collection of data points aggregated together because of certain similarities. For image segmentation, clusters here are different image colors/values.


First, we need to flatten the array into 2 dimensions.

In [ ]:
# Convert MxNx3 image into Kx3 where K=MxN
pixel_values  = img.reshape((-1,3))  #-1 reshape means, in this case MxN

#We convert the unit8 values to float as it is a requirement of the k-means method of OpenCV
pixel_values = np.float32(pixel_values)

Define criteria, number of clusters and apply k-means: We are going to stop the algorithm either when some number of iterations is exceeded (say 100), or if the clusters move less than some epsilon value (let's pick 0.2 here).

When this criterion is satisfied, the algorithm iteration stops.

1. cv.TERM_CRITERIA_EPS — stop the algorithm iteration if specified accuracy, epsilon, is reached.
2. cv.TERM_CRITERIA_MAX_ITER — stop the algorithm after the specified number of iterations, max_iter.
3. cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER — stop the iteration when any of the above condition is met.

The below code defines the stopping criteria in OpenCV:

In [ ]:
# define stopping criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

Set numbers of clusters according to the objects in the.

In [ ]:
# number of clusters (K)
k = 4

attempts = 10
_, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)

# convert back to 8 bit values
centers = np.uint8(centers)

# flatten the labels array
labels = labels.flatten()

# convert all pixels to the color of the centroids
segmented_image = centers[labels.flatten()]

# reshape back to the original image dimension
segmented_image = segmented_image.reshape(img.shape)
# show the image
plt.imshow(segmented_image)
plt.show()

In [ ]:
# function to segment using k-means

def segment_image_kmeans(img, k=3, attempts=10): 

    # Convert MxNx3 image into Kx3 where K=MxN
    pixel_values  = img.reshape((-1,3))  #-1 reshape means, in this case MxN

    #We convert the unit8 values to float as it is a requirement of the k-means method of OpenCV
    pixel_values = np.float32(pixel_values)

    # define stopping criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, attempts, cv2.KMEANS_RANDOM_CENTERS)
    
    # convert back to 8 bit values
    centers = np.uint8(centers)

    # flatten the labels array
    labels = labels.flatten()
    
    # convert all pixels to the color of the centroids
    segmented_image = centers[labels.flatten()]
    
    # reshape back to the original image dimension
    segmented_image = segmented_image.reshape(img.shape)
    
    return segmented_image, labels, centers

# User interaction

Forms provide an easy way to parameterize code. From a code cell, select Insert → Add form field. When you change the value in a form, the corresponding value in the code will change. 

In [ ]:
#@title String fields

text = 'title' #@param {type:"string"}
dropdown = '1st option' #@param ["1st option", "2nd option", "3rd option"]
text_and_dropdown = 'value' #@param ["1st option", "2nd option", "3rd option"] {allow-input: true}

print(text)
print(dropdown)
print(text_and_dropdown)

In [ ]:
#@title Date fields
date_input = '2018-03-22' #@param {type:"date"}

print(date_input)

In [ ]:
#@title Number fields
number_input = 10.0 #@param {type:"number"}
number_slider = 0 #@param {type:"slider", min:-1, max:1, step:0.1}

integer_input = 10 #@param {type:"integer"}
integer_slider = 1 #@param {type:"slider", min:0, max:100, step:1}

print(number_input)
print(number_slider)

print(integer_input)
print(integer_slider)

**Hiding code**

You can change the view of the form by selecting **View → Show/hide code** or using the toolbar above the selected code cell. You can see both code and the form, just the form, or just the code.

In [ ]:
#@title Click `Show code` in the code cell. { display-mode: "form" }

option1 = 'A' #@param ["A", "B", "C"]
print('You selected', option1)

In [ ]:
#@title UPLOAD IMAGES HERE: RUN ME.  { display-mode: "form" }
from google.colab import files

try:
  uploaded = files.upload()
except:
  print("")
  print("Please use Chrome, and enable cookies!")
  print("cookie אנא היכנסו דרך דפדפן כרום במחשב, והפעילו גישה לקבצי ")

# lets the user upload the file, and stores the file name in 'file_names'. The files are uploaded under /content/FILE_NAME
file_names = uploaded.keys() 

# Aruco Markers

In [ ]:
# Load Aruco detector
parameters = cv2.aruco.DetectorParameters_create()
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)

# Get Aruco marker
corners, _, _ = cv2.aruco.detectMarkers(image, aruco_dict, parameters=parameters)

# Draw polygon around the marker
int_corners = np.int0(corners)
cv2.polylines(image, int_corners, True, (0, 255, 0), 50)
plt.imshow(image)

# Aruco Area
aruco_area = cv2.contourArea (corners[0])
print('AruCo Area:',aruco_area, 'px')

# Pixel to cm ratio
pixel_cm_ratio = 5*5 / aruco_area# since the AruCo is 5*5 cm, so we devide 25 cm*cm by the number of pixels
print('Ratio - Each pixel is',pixel_cm_ratio, 'cm*cm')

# Satelite

## Setup

In [ ]:
!pip install geemap

import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

import geemap
import pandas as pd
from IPython.display import Image
import ee, datetime
from pylab import *
from matplotlib.pylab import rcParams

In [ ]:
# Load an image of LANDSAT 8 RGB
map1 = geemap.Map()
image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')
# Center the map and display the image.
map1.centerObject(image, zoom=8)
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 3000,
    'opacity': 1.0,
    'gamma': 1.2,
}
map1.addLayer(image, vis_params, "Landsat Vis")
map1

In [ ]:
#get info on the image
image = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318')
props = geemap.image_props(image)
props.getInfo()#call the data to our pc

In [ ]:
# Load a Landsat 8 image-do NDVI
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
# Compute the NDVI using an expression.
NDVI = image.expression(
    ' ((NIR - RED) / (NIR + RED))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
}).rename('NDVI');
palette =  ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
map1.centerObject(image, 9)
map1.addLayer(NDVI, {'min': 0, 'max': 1, 'palette':palette}, 'NDVI')
map1

In [ ]:
# sentinel 2 image israel
# initialize our map
map1 = geemap.Map()
AOI = ee.Geometry.Point(35.054342198020024, 31.35047863722117)
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2019-01-01","2019-12-31").filterBounds(AOI).first()
#False color imagery is displayed in a combination of standard near 
# infra-red, red and green band. False color composite using near infrared,
# red and green bands is very popular. It is most commonly used to assess plant density
# and healht, as plants reflect near infrared and green light, while absorbing red. 
# Since they reflect more near infrared than green, plant-covered land appears deep
#  red. Denser plant growth is darker red.
# Cities and exposed ground are gray or tan, and water appears blue or black.
rgb = ['B8','B4','B3']
# set some thresholds
rgbViz = {"min":0.0, "max":3000,"bands":rgb}
# initialize our map
     


map1.centerObject(AOI, 7)
map1.addLayer(se2, rgbViz, "S2")
map1.addLayerControl()
map1

In [ ]:
#https://medium.com/@melqkiades/water-detection-using-ndwi-on-google-earth-engine-2919a9bf1951
#https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndwi/
NDWI = se2.normalizedDifference([ 'B3','B8',]).rename('NDWI')
palette =  ['red', 'yellow', 'green', 'cyan', 'blue']
map1.centerObject(se2, 9)
map1.addLayer(NDWI, {'min': -1, 'max': 1, 'palette':palette}, 'NDWI')
map1

In [ ]:
#https://medium.com/@melqkiades/water-detection-using-ndwi-on-google-earth-engine-2919a9bf1951
#https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndwi/
NDWI = se2.normalizedDifference([ 'B3','B8',]).rename('NDWI')
NDWIThreshold = NDWI.gte(0.0);
NDWIMask = NDWIThreshold.updateMask(NDWIThreshold);
palette =  ['blue']
map1.centerObject(se2, 9)
map1.addLayer(NDWIMask,  {'min': 0, 'max': 1, 'palette':palette}, 'NDWI MASK')
map1

Upload ndvi map of specific area.

In [ ]:
NDVI = se2.expression(
    ' ((NIR - RED) / (NIR + RED))', {
      'NIR': se2.select('B8'),
      'RED': se2.select('B4'),}).rename('NDVI');



palette =  ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
map1.centerObject(se2, 9)
map1.addLayer(NDVI, {'min': 0, 'max': 1, 'palette':palette}, 'NDVI')
map1

Get image from the past. For example, December.

In [ ]:
# get December image, we're using the "avg_rad" band

#se2 = ee.ImageCollection('COPERNICUS/S2').filterDate("2020-05-01","2022-12-31").filterBounds(geometry)#.first()
my_boundary = ee.Geometry.Polygon([ [[35.13008776970655, 31.288173501430748],[35.13008776970655, 31.261764527107875],[35.16304675408155, 31.261764527107875],[35.16304675408155, 31.288173501430748]]] ,proj=None)
#fc = ee.FeatureCollection([ee.Feature(ee.Geometry.Polygon([[-109.05, 41], [-109.05, 37], [-102.05, 37], [-102.05, 41])
#viirs2017_12 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2017-12-01","2017-12-31").select('avg_rad').first()
#polygon2 = ee.Geometry.Polygon([
#  [[-122.06197,37.04748], [-122.1830,37.4748], [-122.1830,37.8032], [-122.6197,37.8032], [-122.061974,37.047484]]]);
tls = ee.Feature(ee.FeatureCollection(my_boundary).geometry())

NDVI_clip = NDVI.clip(tls)
map1 = geemap.Map()
map1.centerObject(NDVI_clip, zoom=8)
map1.add_basemap('SATELLITE')
#map1.addLayer(X.select('NDVI'), {})
map1.addLayer(NDVI_clip.select('NDVI'), {'min': 0, 'max': 1, 'palette':palette}, 'NDVI')
#Map.addLayer(X, {'min': 0, 'max': 1, 'palette':palette}, 'NDVI')
map1.addLayerControl()
map1

Exporting the image.

In [ ]:
# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': NDVI_clip,
    'description': 'imageToDriveExample',
    'folder':'Example_folder',
    'scale': 10,
    'region': my_boundary.getInfo()['coordinates']
})
task.start()

In [ ]:
import rasterio
dataset = rasterio.open('/content/imageToDriveExample.tif')
dataset.crs
dataset.profile
from rasterio.plot import show
show(dataset)

band1 = dataset.read([1])
type(band1)

import geemap.colormaps as cm
cm.list_colormaps()

More image options.

In [ ]:
# sentinel 2 image israel
AOI = ee.Geometry.Point(35.570035433516296, 33.0444322996468)
se2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2019-09-01','2020-11-01').filterBounds(AOI).first()
#se2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2019-06-01','2020-11-01').filterBounds(AOI).first()#with cloud

# set some thresholds
rgb = ['B4','B3','B2']
# set some thresholds
rgbViz = {"min":0.0, "max":3000,"bands":rgb}
# initialize our map
     

# initialize our map
map1 = geemap.Map()
map1.centerObject(AOI, 7)
map1.addLayer(se2, rgbViz, "S2")
map1.addLayerControl()
map1

## Cloud masking

In [ ]:
#AOI = ee.Geometry.Point(-122.269, 45.701)
AOI = ee.Geometry.Point(35.570035433516296, 33.0444322996468)
start_date = '2019-06-01'
end_date = '2020-09-01'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 50

Build sentinel-2 collection.

In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))
#Apply the `get_s2_sr_cld_col` function to build a collection according to the parameters defined above.
START_DATE =start_date
END_DATE =end_date
s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

# Cloud components

#Define a function to add the s2cloudless probability layer and derived cloud mask as bands to an S2 SR image input.

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))
#### Cloud shadow components

#Define a function to add dark pixels, cloud projection, and identified 
#shadows as bands to an S2 SR image input. Note that the image input needs to be the result of the above `add_cloud_bands` 
#function because it relies on knowing which pixels are considered cloudy (`'clouds'` band)
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))
    
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

Define cloud mask application function

In [ ]:
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
import numpy as np
import datetime
#35.62093419073687, 33.10805948139925 crop
#35.609261217104056, 33.101876261159404 swemp
point = {'type':'Point', 'coordinates':[ 35.62093419073687, 33.10805948139925]};
#point = {'type':'Point', 'coordinates':[31.35047863722117, 35.054342198020024]};
info = s2_sr.getRegion(point,10).getInfo()
# Reshape image collection 
header = info[0]
data = np.array(info[1:])
data

In [ ]:
# Reshape image collection 
header = info[0]
data = np.array(info[1:])

iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]

# List of used image bands
#band_list = ['B4',u'B8']
band_list = ['B4','B8']
iBands = [header.index(b) for b in band_list]
yData = data[0:,iBands].astype(np.float)

# Calculate NDVI
red = yData[:,0]
nir = yData[:,1]
ndvi = (nir - red) / (nir + red)

In [ ]:
df = pd.DataFrame(data=ndvi, index=list(range(len(ndvi))), columns=['NDVI'])
df = df.interpolate()
df['Date'] = pd.Series(time, index=df.index)

df

In [ ]:
s2_sr_median = s2_sr.median()

In [ ]:
Map = geemap.Map()
# Load an image.
# Center the map and display the image.
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 3000,
    'opacity': 1,
    'gamma': 1.2,
}
Map.addLayer(s2_sr_median, vis_params, "Vis")
Map
#its wil take time to load

# Machine learning models for image classification

## TensorFlow

In [ ]:
#import libs

import tensorflow as tf
import tensorflow_hub as hub

import requests
from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#@title Select an Image Classification model

image_size = 224
dynamic_size = False

model_name = "resnet_v2_152" # @param ['efficientnetv2-s', 'efficientnetv2-m', 'efficientnetv2-l', 'efficientnetv2-s-21k', 'efficientnetv2-m-21k', 'efficientnetv2-l-21k', 'efficientnetv2-xl-21k', 'efficientnetv2-b0-21k', 'efficientnetv2-b1-21k', 'efficientnetv2-b2-21k', 'efficientnetv2-b3-21k', 'efficientnetv2-s-21k-ft1k', 'efficientnetv2-m-21k-ft1k', 'efficientnetv2-l-21k-ft1k', 'efficientnetv2-xl-21k-ft1k', 'efficientnetv2-b0-21k-ft1k', 'efficientnetv2-b1-21k-ft1k', 'efficientnetv2-b2-21k-ft1k', 'efficientnetv2-b3-21k-ft1k', 'efficientnetv2-b0', 'efficientnetv2-b1', 'efficientnetv2-b2', 'efficientnetv2-b3', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'bit_s-r50x1', 'inception_v3', 'inception_resnet_v2', 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152', 'nasnet_large', 'nasnet_mobile', 'pnasnet_large', 'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

model_handle_map = {
  "efficientnetv2-s": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_s/classification/2",
  "efficientnetv2-m": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_m/classification/2",
  "efficientnetv2-l": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/classification/2",
  "efficientnetv2-s-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/classification/2",
  "efficientnetv2-m-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_m/classification/2",
  "efficientnetv2-l-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_l/classification/2",
  "efficientnetv2-xl-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/classification/2",
  "efficientnetv2-b0-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b0/classification/2",
  "efficientnetv2-b1-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b1/classification/2",
  "efficientnetv2-b2-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b2/classification/2",
  "efficientnetv2-b3-21k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/classification/2",
  "efficientnetv2-s-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_s/classification/2",
  "efficientnetv2-m-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_m/classification/2",
  "efficientnetv2-l-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_l/classification/2",
  "efficientnetv2-xl-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/classification/2",
  "efficientnetv2-b0-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/classification/2",
  "efficientnetv2-b1-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b1/classification/2",
  "efficientnetv2-b2-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b2/classification/2",
  "efficientnetv2-b3-21k-ft1k": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b3/classification/2",
  "efficientnetv2-b0": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/classification/2",
  "efficientnetv2-b1": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b1/classification/2",
  "efficientnetv2-b2": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b2/classification/2",
  "efficientnetv2-b3": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/classification/2",
  "efficientnet_b0": "https://tfhub.dev/tensorflow/efficientnet/b0/classification/1",
  "efficientnet_b1": "https://tfhub.dev/tensorflow/efficientnet/b1/classification/1",
  "efficientnet_b2": "https://tfhub.dev/tensorflow/efficientnet/b2/classification/1",
  "efficientnet_b3": "https://tfhub.dev/tensorflow/efficientnet/b3/classification/1",
  "efficientnet_b4": "https://tfhub.dev/tensorflow/efficientnet/b4/classification/1",
  "efficientnet_b5": "https://tfhub.dev/tensorflow/efficientnet/b5/classification/1",
  "efficientnet_b6": "https://tfhub.dev/tensorflow/efficientnet/b6/classification/1",
  "efficientnet_b7": "https://tfhub.dev/tensorflow/efficientnet/b7/classification/1",
  "bit_s-r50x1": "https://tfhub.dev/google/bit/s-r50x1/ilsvrc2012_classification/1",
  "inception_v3": "https://tfhub.dev/google/imagenet/inception_v3/classification/4",
  "inception_resnet_v2": "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4",
  "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/classification/4",
  "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/classification/4",
  "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/classification/4",
  "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/classification/4",
  "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/classification/4",
  "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/4",
  "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/classification/4",
  "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/classification/4",
  "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/classification/4",
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/classification/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/classification/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/classification/5",
}

model_image_size_map = {
  "efficientnetv2-s": 384,
  "efficientnetv2-m": 480,
  "efficientnetv2-l": 480,
  "efficientnetv2-b0": 224,
  "efficientnetv2-b1": 240,
  "efficientnetv2-b2": 260,
  "efficientnetv2-b3": 300,
  "efficientnetv2-s-21k": 384,
  "efficientnetv2-m-21k": 480,
  "efficientnetv2-l-21k": 480,
  "efficientnetv2-xl-21k": 512,
  "efficientnetv2-b0-21k": 224,
  "efficientnetv2-b1-21k": 240,
  "efficientnetv2-b2-21k": 260,
  "efficientnetv2-b3-21k": 300,
  "efficientnetv2-s-21k-ft1k": 384,
  "efficientnetv2-m-21k-ft1k": 480,
  "efficientnetv2-l-21k-ft1k": 480,
  "efficientnetv2-xl-21k-ft1k": 512,
  "efficientnetv2-b0-21k-ft1k": 224,
  "efficientnetv2-b1-21k-ft1k": 240,
  "efficientnetv2-b2-21k-ft1k": 260,
  "efficientnetv2-b3-21k-ft1k": 300, 
  "efficientnet_b0": 224,
  "efficientnet_b1": 240,
  "efficientnet_b2": 260,
  "efficientnet_b3": 300,
  "efficientnet_b4": 380,
  "efficientnet_b5": 456,
  "efficientnet_b6": 528,
  "efficientnet_b7": 600,
  "inception_v3": 299,
  "inception_resnet_v2": 299,
  "mobilenet_v2_100_224": 224,
  "mobilenet_v2_130_224": 224,
  "mobilenet_v2_140_224": 224,
  "nasnet_large": 331,
  "nasnet_mobile": 224,
  "pnasnet_large": 331,
  "resnet_v1_50": 224,
  "resnet_v1_101": 224,
  "resnet_v1_152": 224,
  "resnet_v2_50": 224,
  "resnet_v2_101": 224,
  "resnet_v2_152": 224,
  "mobilenet_v3_small_100_224": 224,
  "mobilenet_v3_small_075_224": 224,
  "mobilenet_v3_large_100_224": 224,
  "mobilenet_v3_large_075_224": 224,
}

model_handle = model_handle_map[model_name]

print(f"Selected model: {model_name} : {model_handle}")


max_dynamic_size = 512
if model_name in model_image_size_map:
  image_size = model_image_size_map[model_name]
  dynamic_size = False
  print(f"Images will be converted to {image_size}x{image_size}")
else:
  dynamic_size = True
  print(f"Images will be capped to a max size of {max_dynamic_size}x{max_dynamic_size}")

labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"

#download labels and creates a maps
downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)

classes = []

with open(downloaded_file) as f:
  labels = f.readlines()
  classes = [l.strip() for l in labels]


In [ ]:
classifier = hub.load(model_handle)

input_shape = image.shape
warmup_input = tf.random.uniform(input_shape, 0, 1.0)
%time warmup_logits = classifier(warmup_input).numpy()

In [ ]:
# Run model on image
%time probabilities = tf.nn.softmax(classifier(image)).numpy()

top_5 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:5].numpy()
np_classes = np.array(classes)

# Some models include an additional 'background' class in the predictions, so
# we must account for this when reading the class labels.
includes_background_class = probabilities.shape[1] == 1001

for i, item in enumerate(top_5):
  class_index = item if includes_background_class else item + 1
  line = f'({i+1}) {class_index:4} - {classes[class_index]}: {probabilities[0][top_5][i]}'
  print(line)

show_image(image, '')

## Object detection

In [ ]:
# let's first restart our runtime, since we're going to use a different model (only for class/teaching perposes)
import os
os.kill(os.getpid(), 9)

# instll the dependencies
!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  # install dependencies

# load the model
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
# libs
from google.colab import files
from matplotlib import pyplot as plt
from skimage import io

# upload an image - you can try 'citrus.jpg' from /Class10/images
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
img = io.imread(img_path)
plt.imshow(img)

In [ ]:
# List of links to images
#imgs = ['https://ultralytics.com/images/zidane.jpg']  # batch of images

# Inference
results = model(img) # accepts URL, Filename, PIL, OpenCV, Numpy..

# Results
#results.print()
results.show()  # or .save()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)

In [ ]:
# if you want, you can load an image to the model from a url, like from a image on a public google drive
file_id = '1xDbIZ7oJkdid5c7-NtFAIGEi-zSmaUFL'
gdrive_url = f'https://drive.google.com/u/1/uc?id={file_id}&export=download'

# Inference
results = model(gdrive_url)

# Results
results.show()

In [ ]:
# check if any people were detected in the image
if 'person' in results.pandas().xyxy[0]['name'].values:
  print('Person detected in image!')
else:
  print('No people detected in image.')

## Mask Rcnn

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!dpkg -i libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.*
!pip install -U -qq tensorflow==2.5.0
exit() # Runtime restart required!

In [ ]:
# Notice how the TF version is now 2.5.0
import tensorflow as tf
tf.__version__

In [ ]:
# let's make sure we have a GPU, under device_type
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
# our GPU card is probably Tesla T4 (what we get from Google)
!nvidia-smi

### Creating dataset:

Use https://www.makesense.ai/ for annotation. 
1. Upload plenty of images (you can get them from Google Images) for training, and label them there. Select them with polygons exactly.
2. When finished all images and labels, click on "Actions", and "Export Annotations".
3. Choose "Single file in COCO JSON format". and then "Export". 
4. Call it "train.json".
5. Do steps 1-4, for around 30% of images, and download a file called "val.json".
6. Create a folder called "dataset", the contents should look like this (in the "train" folder you should have at least 20 images, and in your "val" folder you should have at least 10 images:

In [ ]:
# batch rename files script
# use it, if you downloaded a lot of images from google, and you want to rename them from img0 to img100 for example

# Python 3 code to rename multiple
# files in a directory or folder

# importing os module
import os

# Function to rename multiple files
def main():

	folder = "/content/drive/MyDrive/71254_2023/01_Lectures/Class10/mask_rcnn/dataset/test"
	for count, filename in enumerate(os.listdir(folder)):
		dst = f"image{str(count)}.jpg"
		src =f"{folder}/{filename}" # foldername/filename, if .py file is outside folder
		dst =f"{folder}/{dst}"
		
		# rename() function will
		# rename all the files
		os.rename(src, dst)

# Driver Code
if __name__ == '__main__':
	
	# Calling main() function
	main()


In [ ]:
# cloning the MASK-RCNN repo
!git clone https://github.com/kairess/Mask_RCNN

In [ ]:
!git clone https://github.com/deedeeharris/Mask_RCNN

In [ ]:
# import libs
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw

ROOT_DIR = 'Mask_RCNN'

sys.path.append(ROOT_DIR) 
from mrcnn.config import Config
import mrcnn.utils as utils
from mrcnn import visualize
import mrcnn.model as modellib

In [ ]:
# Download the pretrained model
# This will default to sub-directories in your mask_rcnn_dir, but if you want them somewhere else, updated it here.

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

### Configuration

- NAME: weights, tensorboard (Save folder name)
- IMAGES_PER_GPU: (batch size)
- LEARNING_RATE
- NUM_CLASSES: Number of classes to learn (requires background +1)

In [ ]:
class TrainConfig(Config):
    # Give the configuration a recognizable name
    NAME = "custom"

    # Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 5

    LEARNING_RATE = 0.001

    # Number of classes (including background) - IMPORTANT TO CHANGE ACCORDING TO YOUR LABELS IN YOUR JSON
    NUM_CLASSES = 1 + 1  # background + 1 (flowers)

    # All of our training images are 1920x1012
    IMAGE_MIN_DIM = 512
    IMAGE_MAX_DIM = 512
    
    # Matterport originally used resnet101, but I downsized to fit it on my graphics card
    BACKBONE = 'resnet50' # resnet50

    # To be honest, I haven't taken the time to figure out what these do
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    
config = TrainConfig()
config.display()

### Define the dataset

In [ ]:
class CocoLikeDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
                return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

### Create the Training and Validation Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
root_folder = r'/content/drive/MyDrive/71254_2023/01_Lectures/Class10/mask_rcnn'

dataset_train = CocoLikeDataset()
dataset_train.load_data(f'{root_folder}/dataset/train.json', f'{root_folder}/dataset/train/')
dataset_train.prepare()

dataset_val = CocoLikeDataset()
dataset_val.load_data(f'{root_folder}/dataset/val.json', f'{root_folder}/dataset/val/')
dataset_val.prepare()

image_ids = np.random.choice(dataset_train.image_ids, 4)

print('Train', len(dataset_train.image_ids))
print('Validation', len(dataset_val.image_ids))

for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

### Create the Training Model

In [ ]:
model = modellib.MaskRCNN(
    mode="training",
    config=config,
    model_dir=MODEL_DIR)

model.load_weights(
    COCO_MODEL_PATH,
    by_name=True,
    exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

### Training

Train in two stages:

1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass layers='heads' to the train() function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass layers="all to train all layers.

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
start_train = time.time()

model.train(
    dataset_train,
    dataset_val, 
    learning_rate=config.LEARNING_RATE, 
    epochs=30, 
    layers='heads')

end_train = time.time()
minutes = round((end_train - start_train) / 60, 2)

print(f'Training took {minutes} minutes')

### Save trained model to disk

In [ ]:
# download the trained model to disk

import shutil

original = r'/content/Mask_RCNN/logs/custom20221228T1436/mask_rcnn_custom_0030.h5'
target = r'/content/drive/MyDrive/71254_2023/01_Lectures/Class10/mask_rcnn/mask_rcnn_custom_0030_28122022_flowers.h5'

shutil.copyfile(original, target)

### Prepare to run Inference

In [ ]:
# mount drive first
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class InferenceConfig(TrainConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.65 # CHANGE HERE IF YOU WANT

inference_config = InferenceConfig()

# Recreate the model in inference mode
test_model = modellib.MaskRCNN(
    mode="inference", 
    config=inference_config,
    model_dir=MODEL_DIR)

model_path = test_model.find_last()
print(model_path)

test_model.load_weights(model_path, by_name=True)

### Run inference

In [ ]:
import skimage

mask_colors = [
    (0., 0., 0.), # Background
    (1., 0., 0.), # Red
    (0., 1., 0.)  # Green
]

real_test_dir = f'{root_folder}/dataset/test'
image_paths = []

for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths:
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)

    results = test_model.detect([img_arr], verbose=1)
    r = results[0]

    colors = tuple(np.take(mask_colors, r['class_ids'], axis=0))

    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                dataset_val.class_names, r['scores'], figsize=(16, 8),
                                colors=colors)

### Run on one image

In [ ]:
# libs
from google.colab import files
from matplotlib import pyplot as plt
from skimage import io

# upload an image 
uploaded = files.upload()
img_path = list(uploaded.keys())[0]
img = io.imread(img_path)
plt.imshow(img)

In [ ]:
# or download from URL using wget
!wget https://www.seipasa.com/files/images/img_flor-de-tomate.jpg -O flower.jpg
img = io.imread('flower.jpg')
plt.imshow(img)

In [ ]:
import skimage

mask_colors = [
    (0., 0., 0.), # Background (class 0)
    (1., 0., 0.)#, # Flower (class 1)
    #(0., 1., 0.)  # Add here, for other classes
]


img_arr = np.array(img)

results = test_model.detect([img_arr], verbose=1)
r = results[0]

colors = tuple(np.take(mask_colors, r['class_ids'], axis=0))

visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], figsize=(16, 8),
                            colors=colors)

In [ ]:
# playing around with the results.
# the results is a dictionary
r.keys()

In [ ]:
# let's access the dictionary values
masks = r.get("masks")
class_ids = r.get("class_ids")
scores = r.get("scores")

In [ ]:
# these are the scores (remember, we could of changed the minimum score up top)
scores

In [ ]:
# displaying a mask for example
plt.imshow(masks[:,:,5])

In [ ]:
# let's try to mask out the backgruond
total_mask = masks[:,:,0].copy()
for i in range(len(scores)):
  total_mask = total_mask + masks[:,:,i]
plt.imshow(total_mask)

In [ ]:
# convert mask shape from (500,840) to (500,840,3) 
threeD_mask = np.stack((total_mask, total_mask, total_mask),axis=-1)

In [ ]:
# display the flowers segmented
plt.imshow(threeD_mask*img)